# APRIORI ALGORITHM

In [2]:
# (c) 2016 Everaldo Aguiar & Reid Johnson
#
# Modified from:
# Marcel Caraciolo (https://gist.github.com/marcelcaraciolo/1423287)
#
# Functions to compute and extract association rules from a given frequent itemset 
# generated by the Apriori algorithm.
#
# The Apriori algorithm is defined by Agrawal and Srikant in:
# Fast algorithms for mining association rules
# Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994
import csv
import numpy as np

def load_dataset(filename):
    '''Loads an example of market basket transactions from a provided csv file.

    Returns: A list (database) of lists (transactions). Each element of a transaction is 
    an item.
    '''

    with open(filename,'r') as dest_f:
        data_iter = csv.reader(dest_f, delimiter = ',', quotechar = '"')
        data = [data for data in data_iter]
        data_array = np.asarray(data)
        
    return data_array

def apriori(dataset, min_support=0.5, verbose=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate 
        candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.

    References
    ----------
    .. [1] R. Agrawal, R. Srikant, "Fast Algorithms for Mining Association 
           Rules", 1994.

    """
    C1 = create_candidates(dataset)
    D = list(map(set, dataset))
    F1, support_data = support_prune(D, C1, min_support, verbose=False) # prune candidate 1-itemsets
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        Fk, supK = support_prune(D, Ck, min_support) # prune candidate itemsets
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    if verbose:
        # Print a list of all the frequent itemsets.
        for kset in F:
            for item in kset:
                print("" \
                    + "{" \
                    + "".join(str(i) + ", " for i in iter(item)).rstrip(', ') \
                    + "}" \
                    + ":  sup = " + str(round(support_data[item], 3)))

    return F, support_data

def create_candidates(dataset, verbose=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()

    if verbose:
        # Print a list of all the candidate items.
        print("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    return list(map(frozenset, c1))

def support_prune(dataset, candidates, min_support, verbose=False):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    sscnt = {} # set for support counts
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset)) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    for key in sscnt:
        # Calculate the support of itemset key.
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if verbose:
        for kset in retlist:
            for item in kset:
                print("{" + str(item) + "}")
        print("")
        for key in sscnt:
            print("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets being evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            a.sort()
            b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return retList

def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.
    """
    m = len(H[0])
    if m == 1:
        Hmp1 = calc_confidence(freq_set, H, support_data, rules, min_confidence, verbose)
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1, support_data, rules, min_confidence, verbose)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence, verbose)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:
            rules.append((freq_set - conseq, conseq, conf, support_data[freq_set]))
            pruned_H.append(conseq)

            if verbose:
                print("" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence=0.5, verbose=True):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([itemset]) for itemset in freq_set]
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence, verbose)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, verbose)
    
    return rules

In [3]:
dataset = load_dataset('grocery.csv')
D = list(map(set, dataset))

/Users/shridatta/Downloads/ENTER/envs/cenv/lib/python3.9/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [4]:
type(dataset)

numpy.ndarray

In [5]:
dataset.shape

(9835,)

In [6]:
dataset[:5]

array([list(['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups']),
       list(['tropical fruit', 'yogurt', 'coffee']), list(['whole milk']),
       list(['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads']),
       list(['other vegetables', 'whole milk', 'condensed milk', 'long life bakery product'])],
      dtype=object)

# Complete the assignment below by making use of the provided funtions. 

In [7]:
c1 = create_candidates(dataset, verbose=True) # 1-itemsets

{Instant food products, UHT-milk, abrasive cleaner, artif. sweetener, baby cosmetics, baby food, bags, baking powder, bathroom cleaner, beef, berries, beverages, bottled beer, bottled water, brandy, brown bread, butter, butter milk, cake bar, candles, candy, canned beer, canned fish, canned fruit, canned vegetables, cat food, cereals, chewing gum, chicken, chocolate, chocolate marshmallow, citrus fruit, cleaner, cling film/bags, cocoa drinks, coffee, condensed milk, cooking chocolate, cookware, cream, cream cheese , curd, curd cheese, decalcifier, dental care, dessert, detergent, dish cleaner, dishes, dog food, domestic eggs, female sanitary products, finished products, fish, flour, flower (seeds), flower soil/fertilizer, frankfurter, frozen chicken, frozen dessert, frozen fish, frozen fruits, frozen meals, frozen potato products, frozen vegetables, fruit/vegetable juice, grapes, hair spray, ham, hamburger meat, hard cheese, herbs, honey, house keeping products, hygiene articles, ice c

In [8]:
f1, data = support_prune(D, c1, 0.05, verbose=True)

{domestic eggs}
{whipped/sour cream}
{pork}
{napkins}
{shopping bags}
{brown bread}
{sausage}
{canned beer}
{root vegetables}
{pastry}
{newspapers}
{fruit/vegetable juice}
{soda}
{frankfurter}
{beef}
{curd}
{bottled water}
{bottled beer}
{rolls/buns}
{butter}
{other vegetables}
{pip fruit}
{whole milk}
{yogurt}
{tropical fruit}
{coffee}
{margarine}
{citrus fruit}

{citrus fruit}:  sup = 0.08276563294356888
{margarine}:  sup = 0.05856634468734113
{ready soups}:  sup = 0.0018301982714794102
{semi-finished bread}:  sup = 0.017691916624300967
{coffee}:  sup = 0.05805795627859685
{tropical fruit}:  sup = 0.10493136756481952
{yogurt}:  sup = 0.13950177935943062
{whole milk}:  sup = 0.25551601423487547
{cream cheese}:  sup = 0.03965429588205389
{meat spreads}:  sup = 0.004270462633451958
{pip fruit}:  sup = 0.07564819522114896
{condensed milk}:  sup = 0.010269445856634469
{long life bakery product}:  sup = 0.037417386883579054
{other vegetables}:  sup = 0.1934926283680732
{abrasive cleaner}: 

In [9]:
f, a_data = apriori(dataset, 0.02, verbose=True)

{meat}:  sup = 0.026
{sliced cheese}:  sup = 0.025
{onions}:  sup = 0.031
{frozen meals}:  sup = 0.028
{specialty chocolate}:  sup = 0.03
{frozen vegetables}:  sup = 0.048
{ice cream}:  sup = 0.025
{oil}:  sup = 0.028
{chewing gum}:  sup = 0.021
{ham}:  sup = 0.026
{cat food}:  sup = 0.023
{hard cheese}:  sup = 0.025
{misc. beverages}:  sup = 0.028
{domestic eggs}:  sup = 0.063
{dessert}:  sup = 0.037
{grapes}:  sup = 0.022
{whipped/sour cream}:  sup = 0.072
{pork}:  sup = 0.058
{berries}:  sup = 0.033
{napkins}:  sup = 0.052
{hygiene articles}:  sup = 0.033
{hamburger meat}:  sup = 0.033
{beverages}:  sup = 0.026
{shopping bags}:  sup = 0.099
{brown bread}:  sup = 0.065
{sausage}:  sup = 0.094
{canned beer}:  sup = 0.078
{waffles}:  sup = 0.038
{salty snack}:  sup = 0.038
{root vegetables}:  sup = 0.109
{candy}:  sup = 0.03
{pastry}:  sup = 0.089
{butter milk}:  sup = 0.028
{specialty bar}:  sup = 0.027
{sugar}:  sup = 0.034
{newspapers}:  sup = 0.08
{fruit/vegetable juice}:  sup = 0.

In [10]:
# generating rules based on minimum confidence.
ar = generate_rules(f, a_data, min_confidence=0.3, verbose=True)

{yogurt} ---> {other vegetables}:  conf = 0.311, sup = 0.043
{pip fruit} ---> {other vegetables}:  conf = 0.345, sup = 0.026
{citrus fruit} ---> {other vegetables}:  conf = 0.349, sup = 0.029
{fruit/vegetable juice} ---> {whole milk}:  conf = 0.368, sup = 0.027
{frankfurter} ---> {whole milk}:  conf = 0.348, sup = 0.021
{newspapers} ---> {whole milk}:  conf = 0.343, sup = 0.027
{margarine} ---> {whole milk}:  conf = 0.413, sup = 0.024
{pip fruit} ---> {whole milk}:  conf = 0.398, sup = 0.03
{rolls/buns} ---> {whole milk}:  conf = 0.308, sup = 0.057
{beef} ---> {whole milk}:  conf = 0.405, sup = 0.021
{sausage} ---> {whole milk}:  conf = 0.318, sup = 0.03
{frozen vegetables} ---> {whole milk}:  conf = 0.425, sup = 0.02
{domestic eggs} ---> {other vegetables}:  conf = 0.351, sup = 0.022
{butter} ---> {other vegetables}:  conf = 0.361, sup = 0.02
{pastry} ---> {whole milk}:  conf = 0.374, sup = 0.033
{brown bread} ---> {whole milk}:  conf = 0.389, sup = 0.025
{domestic eggs} ---> {whole m

# Comments on the results of Apriori Algorithm.

Using the apriori algorithm we have generated some rules that have confidence more than the minimum confidence. These association rules describe that these items occur together. In other words, if I were to shop domestic eggs, then there is a 47.3% (conf = 0.473) chance of me also buying whole milk. 

These types of association rules are used by Amazon, eBay, and other ecommerce websites to suggest what people bought with the item you are currently buying. 

There are also other interesting uses. For example, IKEA uses this consumer data to build new stores' layout in a way that arranges the items with strong confidence in association rules near each other to instill the thought of buying in the customers. 

This has hugely increased the profits for the company and is now incorporated by every store around the world.

# PART - 3: INTEREST FACTOR

In [12]:
int_factor = {}
for item in ar:
    print("A:", item[0], "B:", item[1], "S[AB]:", item[3], "S[A]:", a_data[item[0]], "S[B]:", a_data[item[1]])
    int_factor[item[0]] = [item[1], round(item[3]/(a_data[item[1]]*a_data[item[0]]), 3)]
# int_factor

A: frozenset({'yogurt'}) B: frozenset({'other vegetables'}) S[AB]: 0.04341637010676157 S[A]: 0.13950177935943062 S[B]: 0.1934926283680732
A: frozenset({'pip fruit'}) B: frozenset({'other vegetables'}) S[AB]: 0.026131164209456024 S[A]: 0.07564819522114896 S[B]: 0.1934926283680732
A: frozenset({'citrus fruit'}) B: frozenset({'other vegetables'}) S[AB]: 0.02887646161667514 S[A]: 0.08276563294356888 S[B]: 0.1934926283680732
A: frozenset({'fruit/vegetable juice'}) B: frozenset({'whole milk'}) S[AB]: 0.026639552618200304 S[A]: 0.0722928317234367 S[B]: 0.25551601423487547
A: frozenset({'frankfurter'}) B: frozenset({'whole milk'}) S[AB]: 0.020538891713268937 S[A]: 0.058973055414336555 S[B]: 0.25551601423487547
A: frozenset({'newspapers'}) B: frozenset({'whole milk'}) S[AB]: 0.027351296390442297 S[A]: 0.07981698017285206 S[B]: 0.25551601423487547
A: frozenset({'margarine'}) B: frozenset({'whole milk'}) S[AB]: 0.024199288256227757 S[A]: 0.05856634468734113 S[B]: 0.25551601423487547
A: frozenset(

In [12]:
print("TOP-5 rules when sorted by Support: ")
ar_sorted_support = sorted(ar, key=lambda x: x[3], reverse=True)
for i in range(5):
    print(ar_sorted_support[i][0], "--->", ar_sorted_support[i][1], ": support = ", round(ar_sorted_support[i][3], 3))


print("\n")

print("TOP-5 rules when sorted by Confidence: ")
ar_sorted = sorted(ar, key=lambda x: x[2], reverse=True)
for i in range(5):
    print(ar_sorted[i][0], "--->", ar_sorted[i][1], ": conf = ", ar_sorted[i][2])
    
print("\n")
    
print("TOP-5 rules when sorted by Interest Factor: ")
int_factor_sorted = sorted(int_factor.items(), key=lambda x: x[1][1], reverse=True)
# print(int_factor_sorted)
for i in range(5):
    print(int_factor_sorted[i][0], "--->", int_factor_sorted[i][1][0], ": interest_factor = ", int_factor_sorted[i][1][1])

TOP-5 rules when sorted by Support: 
frozenset({'other vegetables'}) ---> frozenset({'whole milk'}) : support =  0.075
frozenset({'rolls/buns'}) ---> frozenset({'whole milk'}) : support =  0.057
frozenset({'yogurt'}) ---> frozenset({'whole milk'}) : support =  0.056
frozenset({'root vegetables'}) ---> frozenset({'whole milk'}) : support =  0.049
frozenset({'root vegetables'}) ---> frozenset({'other vegetables'}) : support =  0.047


TOP-5 rules when sorted by Confidence: 
frozenset({'other vegetables', 'yogurt'}) ---> frozenset({'whole milk'}) : conf =  0.5128805620608898
frozenset({'butter'}) ---> frozenset({'whole milk'}) : conf =  0.4972477064220184
frozenset({'curd'}) ---> frozenset({'whole milk'}) : conf =  0.4904580152671756
frozenset({'root vegetables', 'other vegetables'}) ---> frozenset({'whole milk'}) : conf =  0.4892703862660944
frozenset({'root vegetables', 'whole milk'}) ---> frozenset({'other vegetables'}) : conf =  0.47401247401247404


TOP-5 rules when sorted by Interes

In the above cell, we calculated the Top5 rules based on support, confidence, and interest factor.

The highest support is for 'other vegetables' and 'whole milk'. These exist as a part of the association rule in 3 out of the 5 highest confidence rules and in 2 out of 5 highest interest factor rules. This shows that if a person buys whole milk, they are very likely to buy other vegetables and vice versa.